In [19]:
# !pip install transformers --upgrade
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import os
import pandas as pd
import datasets
import gc
from tqdm import tqdm

In [2]:
model_ckpt = 'Helsinki-NLP/opus-mt-ko-en'
# model_ckpt2 = 'alphahg/opus-mt-ko-en-finetuned-ko-to-en100'
# pipe2 = pipeline('translation', model=model_ckpt2, device=0)

In [3]:
pipe = pipeline('translation', model = model_ckpt, device=0)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/822k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/794k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

In [4]:
# test
pipe('인간 바둑기사가 인공지능에 승리할 수 있었던 이유는 인공지능으로부터 전술을 직접 물려 받았기 때문이다.')

[{'translation_text': 'The reason the human GO driver was able to win AI is because he inherited a strategy directly from AI.'}]

---
# kor -> eng backtranslation 작업 순서

### 작업 할당 :

- `김시혁` :  'jd' 전라도
- `정선아` :  'kw' 강원도
- `임종인` :  'gs' 경상도
- `이승용` :  'jj' 제주도
- `신덕식` :  'cc' 충청도

---

### `data_save(dataset, pipe, '지역', 작업크기, 시작index)`
>1. ### `dataset` : 데이터 셋 [dataframe : pandas dataframe]
>2. ### `pipe` : hugging face pipeline [func : huggingface pipeline function]
>3. ### `지역` : 사투리 지역 [str : 'jj', 'gs', 'kw', 'cc', 'jd']
>4. ### `작업크기` : num of rows [int]
>5. ### `directory` : 저장 경로 [str : directory]
>>예) data_save(df,pipe,'jj',5000,0,'/aiffel/aiffelthon/') -> '지역', 시작index만 변경해주시면 됩니다.

In [14]:
directory = os.getenv('HOME') + '/aiffel/aiffelthon/' # 데이터 경로 지정

df = pd.read_csv(directory+'dataset_full_v1.csv') # 데이터 파일 불러오기
df.drop_duplicates(inplace=True)
df['text'] = df['text'].str.replace('[\(\)]','',regex=True)
df['dial'] = df['dial'].str.replace('[\(\)]','',regex=True)

In [15]:
def data_save(data, pipe, reg, num_rows, directory) :
    '''
    1. ### `dataset` : 데이터 셋 [dataframe : pandas dataframe]
    2. ### `pipe` : hugging face pipeline [func : huggingface pipeline function]
    3. ### `지역` : 사투리 지역 [str : 'jj', 'gs', 'kw', 'cc', 'jd']
    4. ### `작업크기` : num of rows [int]
    5. ### `directory` : 저장 경로 [str : directory]
        예) data_save(df,pipe,'jj',5000,0,'/aiffel/aiffelthon/') -> '지역', 시작index만 변경해주시면 됩니다.
    '''
    data = data.loc[data['reg'] == reg].sample(frac=1)3
    
    df_final = pd.DataFrame(columns=['text', 'dial', 'reg', 'pair','eng'])

    for i in range(0,len(data), num_rows) :
        dataset = data[i : i +num_rows].reset_index()
        dataset = datasets.Dataset.from_pandas(dataset)
        dataset = dataset.map(lambda ds : {'eng' : pipe(ds['text'])[0]['translation_text']})
        dataset.set_format('pandas')
        df_final = df_final.merge(dataset[:], how='outer')
        df_final.to_csv(directory + f'dataset_en_ko_{reg}_{i}_{i+num_rows}.csv', index=False)
        print(f'작업 완료 : {reg}_{i}_{i+num_rows}' )
        print(f'저장 경로 : {directory}dataset_en_ko_{reg}_{i}_{i+num_rows}.csv')

In [16]:
reg = 'kw' # 지역
num_rows = 10000

In [17]:
kw = data_save(df,pipe,reg,num_rows,directory)

  0%|          | 0/10000 [00:00<?, ?ex/s]

/opt/conda/lib/python3.9/site-packages/transformers/pipelines/base.py:899: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


작업 완료 : kw_0_10000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_en_ko_kw_0_10000.csv


  0%|          | 0/10000 [00:00<?, ?ex/s]

작업 완료 : kw_10000_20000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_en_ko_kw_10000_20000.csv


...

작업 완료 : kw_360000_370000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_en_ko_kw_360000_370000.csv


  0%|          | 0/10000 [00:00<?, ?ex/s]

작업 완료 : kw_370000_380000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_en_ko_kw_370000_380000.csv


  0%|          | 0/10000 [00:00<?, ?ex/s]

KeyboardInterrupt: 

In [21]:
# data merge하기
import glob
files = glob.glob(directory+f'*{reg}*.csv')
df_final = pd.DataFrame(columns=['text', 'dial', 'reg', 'pair','eng'])
for i in files :
    df_new = pd.read_csv(i)
    df_final = df_final.merge(df_new, how='outer')
df_final.to_csv(directory + f'datset_{reg}_{num_rows}_final_v{start}.csv' , index=False)
df_final.head()

NameError: name 'start' is not defined